In [1]:
import docx
import re
from langchain_core.documents import Document

### Обработка файла 1
---

In [2]:
def extract_situations_from_docx(file_path):
    document = docx.Document(file_path)
    situations = {}

    current_number = 0
    cur_text = ""
    situations[current_number] = {}

    for para in document.paragraphs:
        numbering = para._p.pPr.numPr if para._p.pPr is not None else None
        para_text = para.text.strip()

        if numbering is not None:
            
            situations[current_number]["assessment"] = cur_text
            current_number += 1
            situations[current_number] = {}
            cur_text = ""

            if para_text.startswith("Ситуация:"):
                cur_text += para_text.strip()
        else:
            if para_text.startswith("Оценка критичности, рекомендации:"):
                situations[current_number]["situation"] = cur_text
                cur_text = ""
                cur_text += para_text.strip()
            else:
                cur_text += para_text
    situations[current_number]["assessment"] = cur_text
    situations.pop(0, None)

    return situations

In [3]:
file_path = '../data/1_file.docx'
analysis_situations = extract_situations_from_docx(file_path)

In [4]:
analysis_docs = []
for key in analysis_situations:
    analysis_docs.append(
        Document(
            page_content=f'{analysis_situations[key]["situation"]}\n{analysis_situations[key]["assessment"]}', 
            metadata={
                "file_name": "file_1", 
            },
            id=key
        )
    )

In [43]:
# analysis_docs

### Обработка файла 2
---

In [5]:
def extract_situations_from_docx_2(file_path):
    document = docx.Document(file_path)
    situations = {}

    current_number = 0
    cur_text = ""
    situations[current_number] = {}

    for para in document.paragraphs:
        numbering = para._p.pPr.numPr if para._p.pPr is not None else None
        para_text = para.text.strip()

        if numbering is not None:
            
            situations[current_number]["action"] = cur_text
            current_number += 1
            situations[current_number] = {}
            cur_text = ""

            if para_text.startswith("Ситуация:"):
                cur_text += para_text.strip()
        else:
            if para_text.startswith("Необходимые мероприятия юридического подразделения:"):
                situations[current_number]["situation"] = cur_text
                cur_text = ""
                cur_text += para_text.strip()
            else:
                cur_text += para_text
    situations[current_number]["action"] = cur_text
    situations.pop(0, None)

    return situations

In [6]:
file_path = '../data/2_file.docx'
procedures_situations = extract_situations_from_docx_2(file_path)

In [28]:
# procedures_situations

In [7]:
procedures_docs = []
for key in procedures_situations:
    procedures_docs.append(
        Document(
            page_content=f'{procedures_situations[key]["situation"]}\n{procedures_situations[key]["action"]}', 
            metadata={
                "file_name": "file_2", 
            },
            id=33+key
        )
    )

In [48]:
# procedures_docs

### Credentials

In [9]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

proxy_url = os.environ.get("OPENAI_PROXY_URL")

### Initialization

In [10]:
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings

embedding_function = GigaChatEmbeddings(credentials=os.getenv("GIGACHAT_CREDENTIALS"), verify_ssl_certs=False, model="Embeddings")

In [ ]:
# from langchain_openai import OpenAIEmbeddings

# embedding_function = OpenAIEmbeddings(model="text-embedding-3-large", http_client=httpx.Client(proxy=proxy_url))

### Init chroma db
---

In [11]:
import chromadb
from langchain_chroma import Chroma

persistent_client  = chromadb.PersistentClient(path="chromadb_db/")

In [12]:
vector_store = Chroma(client=persistent_client, embedding_function=embedding_function)

In [14]:
# %%time

# vector_store.add_documents(documents=analysis_docs + procedures_docs)

In [54]:
vector_store.similarity_search("Долг", k=3)

[Document(id='21', metadata={'file_name': 'file_1'}, page_content='Ситуация: Несовпадение сумм задолженности как в разбивке, так и в общем итоге в резолютивной части судебного акта с РЦИ (расчет, расчет цены иска).\nОценка критичности, рекомендации: Некритично, если расхождения сумм задолженности в диапазоне 250 рублей (включительно). В остальных случаях критично.'),
 Document(id='2', metadata={'file_name': 'file_1'}, page_content='Ситуация: Судебный акт вынесен по разным продуктам (кредитная карта + кредит, КД, КК), как следствие, в резолютивной части судебного акта сумма госпошлины (г/п, гп) указана общей суммой без разбивки по продуктам. Оплата госпошлины (г/п, гп) рассчитывается исходя из общей суммы задолженности по продуктам.\nОценка критичности, рекомендации: Некритично. Необходимо произвести расчет госпошлины для каждого обязательства и указать сумму в карточке дела и в запросе в АС ВРМ «Правовое обеспечение».'),
 Document(id='20', metadata={'file_name': 'file_1'}, page_content

In [15]:
vector_store.similarity_search("Долг", k=3)

[Document(id='34', metadata={'file_name': 'file_2'}, page_content='Ситуация: получен отказ в присуждении от ПЦП ОЦ по причине «кредитный договор (КД) на срочных счетах.».\nНеобходимые мероприятия юридического подразделения: необходимо выставить задачу в ПЦП ЦУЗ с комментарием: «Присуждение#Отказ_УСПЗ#_Договор на срочных счетах». Судебных акт необходимо передавать в ПСИП.В случаях возникновения дополнительных вопросов необходимо обратиться к методологам аппарата ТБ.'),
 Document(id='1', metadata={'file_name': 'file_1'}, page_content='Ситуация: Судебный акт вынесен по нескольким договорам, сумма госпошлины (г/п, гп) указана общей суммой без разбивки.\nОценка критичности, рекомендации: Некритично. Необходимо произвести расчет госпошлины для каждого обязательства и указать сумму в карточке дела и в запросе в АС ВРМ «Правовое обеспечение».'),
 Document(id='7', metadata={'file_name': 'file_1'}, page_content='Ситуация: Ошибки в исполнительном документе (в отношении ФЛ (физического листа) – ФИ

In [16]:
from langchain_gigachat.chat_models import GigaChat

giga_llm = GigaChat(
            credentials=os.getenv("GIGACHAT_CREDENTIALS"),
            model="GigaChat-2-Max",
            verify_ssl_certs=False
        )

In [17]:
giga_llm.invoke("Привет")

AIMessage(content='Здравствуйте! Как ваши дела?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 14, 'completion_tokens': 7, 'total_tokens': 21}, 'model_name': 'GigaChat-2-Max:2.0.27.4', 'x_headers': {'x-request-id': '961e621e-117f-41ea-9440-4a9c928d9137', 'x-session-id': '33a35851-3503-4abf-8691-0e5e62533dac', 'x-client-id': None}, 'finish_reason': 'stop'}, id='961e621e-117f-41ea-9440-4a9c928d9137')